In [1]:

import warnings
from flask import  Flask, jsonify, request, render_template
# Suppress all warnings
warnings.filterwarnings("ignore")

from Cache_Load_Data import *

# Restore warnings to default behavior
warnings.resetwarnings()

app = Flask(__name__)


Connected to MySQL Server version  8.0.36


[NbConvertApp] Converting notebook MySQL_Load_Data.ipynb to script
[NbConvertApp] Writing 3855 bytes to MySQL_Load_Data.py


Number of documents: 112036
found
[['Sat Apr 25 12:39:47 +0000 2020', 1254027325475258368, 'RT elmundociencia: Los españoles creen que la "responsabilidad ciudadana" es el factor más importante para controlar la epidemia de corona…', 'Jimenezlosantos', 1, 2, 'es', 0, 0, None, [], False], ['Sat Apr 25 12:30:00 +0000 2020', 1254024861158002691, 'Al 24 de abril, Puebla ocupa el séptimo lugar nacional de los estados con más casos confirmados de coronavirus en M… https://t.co/8HjnFS6Uq0', 'El_Universal_Mx', 0, 0, 'es', 0, 0, None, [], False], ['Sat Apr 25 12:30:00 +0000 2020', 1254024861204328453, 'हमारे देश के वह न्यूज़ चैनल और एंकर \n#Corona से भी ज्यादा खतरनाक हैं जो इसे सांप्रदायिक रंग दे रहे है. कहते हैं कि… https://t.co/aGDQ7Aoe4S', 'nlhindi', 0, 0, 'hi', 0, 0, None, ['Corona'], False], ['Sat Apr 25 12:30:00 +0000 2020', 1254024861866790912, 'कोरोना संकट में ऐसे मनाएं रमजान का त्योहार, बस ध्यान रखें ये जरूरी बातें... \nhttps://t.co/RbEuNR2O1G\n\n#CoronaVirus… https://t.co/gRniFHMJZB',

[NbConvertApp] Converting notebook MongoDB_Load_Data.ipynb to script
[NbConvertApp] Writing 16074 bytes to MongoDB_Load_Data.py
[NbConvertApp] Converting notebook Cache_Load_Data.ipynb to script
[NbConvertApp] Writing 10572 bytes to Cache_Load_Data.py


In [2]:

cache={}
user_cache = {}
tweet_cache = {}
hashtag_cache = {}
filename = r'C:\Users\Yuqing Dai\Desktop\rutgers-ds\database\final project\cache.json'

cache = load_cache(filename)
if len(cache) > 0:
    user_cache  = cache.get('User',{})
    tweet_cache  = cache.get('Tweet',{})
    hashtag_cache  = cache.get('Hashtag',{})
    

    
    
    
def main(Type, string, limit=None, start_time=None, end_time=None):
    
    if Type == 'user':
        result = user_search(user_cache, string, limit)  # Func 5
        print(result)
        if isinstance(result, list):
            n = min(limit, len(result))
            json_data = json.dumps(result[:n])
            return json_data
    elif Type == 'text':
        result = string_search(tweet_cache, start_time, end_time, string, limit)
        print(result)# Func 1
        if isinstance(result, list):
            n = min(limit, len(result))
            json_data = json.dumps(result[:n])
            print(result)
            return json_data
           
        else:
            json_data = json.dumps(result)
            return json_data
    elif Type == 'hashtag':
        result = hashtag_search(hashtag_cache, start_time, end_time, string, limit)  # Func 2
        print(result)
        if isinstance(result, list):
            n = min(limit, len(result))
            json_data = json.dumps(result[:n])
            return json_data
        else:
            json_data = json.dumps(result)
            return json_data
    elif Type == 'author':
        screen_name = string
        result = get_tweets_by_screen_name(screen_name)
        print(result)
        json_data = json.dumps(result)
        return json_data
    elif Type == 'retweet':
        retweet_id = int(string)
        result = find_retweeters_by_tweet_id(retweet_id)
        print(result)
        json_data = json.dumps(result)
        return json_data
    else:
        print("Invalid search type:", Type)






    
    
    
def save_cache_to_disk():
    cache['User']=user_cache
    cache['Tweet']=tweet_cache
    cache['Hashtag']=hashtag_cache
    save_cache(cache,filename)
    
    
    
import time
def periodic_save(interval):
    while True:
        save_cache_to_disk()
        time.sleep(interval)

# Define the interval for periodic saving (e.g., every 10 seconds)
save_interval = 60

# Start the periodic saving process in a separate thread
import threading
save_thread = threading.Thread(target=periodic_save, args=(save_interval,))
save_thread.start()







In [3]:
Type = "author"     
string = 'bablusharmabjp'
json_data = json.dumps(main(Type,string))
print(type(json_data))


[['Sat Apr 18 04:20:02 +0000 2020', '*अंदर से कोई बाहर न जा सके*\n\n*बाहर से कोई अंदर न जा सके*\n\n*घर में रहे सेफ रहे*\n\n#corona  रूपी रावण को मारना है*\n\n#सुप्रभात #हेलोजिंदगी', 6, 1]]
<class 'str'>


In [4]:
def get_top_users():
    conn = None
    try:
        conn = mysql.connector.connect(host='localhost', database='twitterbase', user='root', password='Dyq20150801@')
        cursor = conn.cursor()
        query = "SELECT NAME, FOLLOWERS_COUNT FROM user_info ORDER BY FOLLOWERS_COUNT DESC LIMIT 10"
        cursor.execute(query)
        results = cursor.fetchall()
        return [{'name': row[0], 'followers_count': row[1]} for row in results]
    except Error as e:
        print('Error while connecting to MySQL', e)
        
top_users = get_top_users()
if top_users:
    for user in top_users:
        print(user)
        
def get_top_tweets():
    try:
        client = pymongo.MongoClient('localhost', 27017)
        db = client.db 
        tweets_collection = db.tweets 


        try:
            tweets_collection.create_index([('favorite_count', pymongo.DESCENDING)])
        except pymongo.errors.OperationFailure as e:
            print(f"Error creating index: {e}")


        top_tweets = list(tweets_collection.find({}, {'_id': 0}).sort("favorite_count", pymongo.DESCENDING).limit(10))
        return top_tweets
    except pymongo.errors.PyMongoError as e:
        print(f"MongoDB error: {e}")
        return []


top_tweets = get_top_tweets()
if top_tweets:
    for tweet in top_tweets:
        print(tweet)
else:
    print("No tweets found or there's an error with the database connection.")


{'name': 'detikcom', 'followers_count': 15928061}
{'name': 'J.K. Rowling', 'followers_count': 14608046}
{'name': 'AajTak', 'followers_count': 9706667}
{'name': 'ABP News', 'followers_count': 9563509}
{'name': 'TIMES NOW', 'followers_count': 9499855}
{'name': 'Rajdeep Sardesai', 'followers_count': 8947464}
{'name': 'Le Monde', 'followers_count': 8808784}
{'name': 'tvOneNews', 'followers_count': 8787649}
{'name': 'Kompas.com', 'followers_count': 7678373}
{'name': 'NTV', 'followers_count': 7429200}
{'created_at': 'Fri Mar 13 00:43:40 +0000 2020', 'retweeted_id': 0, 'og_tweet_id': 1238264431320215553, 'text': '*corona virus enters my body*\n\nThe 4 Flintstone gummies I ate in 2005: https://t.co/3STfdIQtaT', 'screen_name': 'HtownBabyG', 'retweet_count': 237307, 'favorite_count': 811062, 'lang': 'en', 'quote_count': 6828, 'reply_count': 1811, 'media': 'http://pbs.twimg.com/ext_tw_video_thumb/1238264358150569986/pu/img/IqZsgAzo4VjQbUF3.jpg', 'hashtags': [], 'is_retweet': False}
{'created_at':

In [5]:
@app.route("/")
def index():
    return render_template('index.html')

@app.route('/search/text', methods=['GET'])
def search_text():
    string = str(request.args.get('string'))
    limit = request.args.get('limit', type=int)
    start_time = str(request.args.get('start_time'))
    end_time = str(request.args.get('end_time'))

    result = main('text', string, limit, start_time, end_time)
    return jsonify(result)

@app.route('/search/hashtag', methods=['GET'])
def search_hashtag():
    string = str(request.args.get('string'))
    limit = request.args.get('limit', type=int)
    start_time = str(request.args.get('start_time'))
    end_time = str(request.args.get('end_time'))

    result = main('hashtag', string, limit, start_time, end_time)
    return jsonify(result)

@app.route('/search/user', methods=['GET'])
def search_user():
    string = str(request.args.get('string'))
    limit = request.args.get('limit', type=int)
    start_time = str(request.args.get('start_time'))
    end_time = str(request.args.get('end_time'))

    result = main('user', string, limit)
    return jsonify(result)

@app.route('/search/author', methods=['GET'])
def search_author():
    string = str(request.args.get('string'))
    result = main('author', string)  
    return jsonify(result)

@app.route('/search/retweet', methods=['GET'])
def search_retweet():
    string = str(request.args.get('string'))
    result = main('retweet', string)  
    return jsonify(result)

@app.route('/top_users', methods=['GET'])
def top_users():
    users = get_top_users()
    return jsonify(users)

@app.route('/top_tweets', methods=['GET'])
def top_tweets():
    tweets = get_top_tweets()
    return jsonify(tweets)


In [6]:
def search(search_type):
    string = str(request.args.get('string'))
    limit = request.args.get('limit', type=int)
    start_time = str(request.args.get('start_time'))
    end_time = str(request.args.get('end_time'))
    
    if search_type in ['text', 'hashtag', 'user']:
        result = main(search_type, string, limit, start_time, end_time)
        return jsonify(result)
    elif search_type in ['author', 'retweet']:
        result = main(search_type, string)
        return jsonify(result)
    else:
        return jsonify({'error': 'Invalid search type'})

In [ ]:
if __name__ == "__main__":
    app.run(debug=True,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


D:\Anaconda\lib\site-packages\werkzeug\serving.py:718: ResourceWarning: unclosed <socket.socket fd=1016, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>
  self.socket = socket.fromfd(fd, address_family, socket.SOCK_STREAM)
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2024 10:19:03] "GET / HTTP/1.1" 200 -
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x178038bdd00>
127.0.0.1 - - [26/Apr/2024 10:19:03] "GET /static/script.js HTTP/1.1" 304 -
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x17803930040>
127.0.0.1 - - [26/Apr/2024 10:19:04] "GET /top_tweets HTTP/1.1" 200 -
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x17803930220>
127.0.0.1 - - [26/Apr/2024 10:19:04] "GET /top_users HTTP/1.1" 200 -
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x178039d21c0>
127.0.0.1 - - [26/Apr/2024 10:19:43] "GET /search/text?string=Trump&limit=10&start_time=12:00:00&end_time

12:00:0015:40:00TrumpLIMIT10
Going to tables
[['Sat Apr 25 12:02:00 +0000 2020', 1254017815142490114, 'Presiden Amerika Serikat Donald Trump mengatakan telah menerima telepon dari Presiden Joko Widodo. \n\nPembicaraan te… https://t.co/SnywFBm8j0', 'KompasTV', 3, 13, 'in', 0, 1, None, [], False], ['Sat Apr 25 12:20:14 +0000 2020', 1254022403396636674, '@MarriedtoBringo Trump is on Twitter every single day, but even I mentioned Corona a week earlier than he did.', 'LabradorYuki', 1, 5, 'en', 0, 2, None, [], False], ['Sat Apr 25 12:02:59 +0000 2020', 1254018062694703106, 'The Trump Admin bought at least 29 mil doses of this poison and promoted it daily through press briefings, \u2066… https://t.co/rKrkvzJh3x', 'PassionFruit62', 4, 4, 'en', 0, 0, None, [], False], ['Sat Apr 25 12:02:21 +0000 2020', 1254017905769005056, '„Nur ein Idiot hätte jetzt keine Angst“, sagt Siri #Hustvedt zur #Coronakrise - und dass Donald #Trump ein Verrückt… https://t.co/b9VcGKL0NH', 'BerlinerNotizen', 3, 4, 'de'

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x178039a2280>
127.0.0.1 - - [26/Apr/2024 10:19:53] "GET /search/text?string=Russia&limit=10&start_time=12:00:00&end_time=15:40:00 HTTP/1.1" 200 -


12:00:0015:40:00RussiaLIMIT10
Going to tables
[['Sat Apr 25 12:26:22 +0000 2020', 1254023949224013824, '@USMCSDI Will Russia didn’t work, neither did impeachment! So let’s  get trump on this corona virus! Soros and the… https://t.co/4hyd70ZxWH', 'davidjdurkin4', 0, 0, 'en', 0, 0, None, [], False], ['Sat Apr 25 12:30:00 +0000 2020', 1254024861321584640, '【利害了他的国】俄罗斯民众因无法上班养家聚集抗议！\n➡️下载链接：https://t.co/c4XoGRPwWs\n\n#PonyVPN #VPN #翻墙 #梯子 #Russia #Vladikavkaz\n#BackToWork… https://t.co/wveKe7dSRY', 'Ponyjsq', 0, 0, 'zh', 0, 0, None, ['PonyVPN', 'VPN', '翻墙', '梯子', 'Russia', 'Vladikavkaz', 'BackToWork'], False], ['Sat Apr 25 12:33:41 +0000 2020', 1254025790297120770, 'Erwartungsgemäß war die gestrige Demo gegen die Schutzmaßnahmen gegen Corona ein Propaganda-Beitrag von Russia Toda… https://t.co/vok03KCNzh', 'Svejk', 0, 0, 'de', 0, 0, None, [], False], ['Sat Apr 25 12:37:28 +0000 2020', 1254026742450266112, 'Erwartungsgemäß war die gestrige Demo gegen die Schutzmaßnahmen gegen Corona ein Pro

...

127.0.0.1 - - [26/Apr/2024 10:22:49] "GET /search/text?string=trump&limit=5&start_time=12:00:00&end_time=15:40:00 HTTP/1.1" 200 -


12:00:0015:40:00trumpLIMIT5
found in Cache
Time taken: -1.7141413673395566e+18 seconds
[['Sat Apr 25 12:02:00 +0000 2020', 1254017815142490114, 'Presiden Amerika Serikat Donald Trump mengatakan telah menerima telepon dari Presiden Joko Widodo. \n\nPembicaraan te… https://t.co/SnywFBm8j0', 'KompasTV', 3, 13, 'in', 0, 1, None, [], False], ['Sat Apr 25 12:20:14 +0000 2020', 1254022403396636674, '@MarriedtoBringo Trump is on Twitter every single day, but even I mentioned Corona a week earlier than he did.', 'LabradorYuki', 1, 5, 'en', 0, 2, None, [], False], ['Sat Apr 25 12:02:59 +0000 2020', 1254018062694703106, 'The Trump Admin bought at least 29 mil doses of this poison and promoted it daily through press briefings, \u2066… https://t.co/rKrkvzJh3x', 'PassionFruit62', 4, 4, 'en', 0, 0, None, [], False], ['Sat Apr 25 12:02:21 +0000 2020', 1254017905769005056, '„Nur ein Idiot hätte jetzt keine Angst“, sagt Siri #Hustvedt zur #Coronakrise - und dass Donald #Trump ein Verrückt… https://t.co/

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x178039fb580>
127.0.0.1 - - [26/Apr/2024 10:22:55] "GET /search/author?string=KompasTV&limit=null&start_time=null&end_time=null HTTP/1.1" 200 -


[['Fri Apr 24 05:02:00 +0000 2020', 'Pemerintah Kabupaten Sragen, Jawa Tengah, meminta tiap desa menyiapkan tempat karantina bagi para pemudik yang memb… https://t.co/Nh6lhuu83A', 108, 52], ['Fri Apr 24 14:29:13 +0000 2020', 'Berikut TOP 3 NEWS - 24 April 2020:\n1. 700 Kendaraan Putar Balik\n2. Andi Taufan Mundur dari Stafsus\n3. Positif Coro… https://t.co/ag2T1DulSE', 72, 18], ['Thu Apr 23 22:39:33 +0000 2020', '4 Harimau dan 3 Singa Afrika di Kebun Binatang Bronx Terinfeksi Virus Corona https://t.co/45DAzFpu2r https://t.co/uZTuRo3LSd', 54, 25], ['Thu Apr 23 09:38:08 +0000 2020', '[FULL] Update Corona 23 April: 7775 Positif, 960 Pasien Sembuh https://t.co/thvquLObdk https://t.co/YjuRgIx6KV', 37, 11], ['Sat Apr 25 08:02:00 +0000 2020', 'Pandemi Corona membawa dampak bagi ribuan pekerja di sektor pariwisata di Bali. Lebih dari 5.000 orang pekerja di s… https://t.co/12hfRthnNF', 26, 13], ['Sat Apr 25 12:02:00 +0000 2020', 'Presiden Amerika Serikat Donald Trump mengatakan telah menerima t

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x178039fb700>
